Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [25]:
from itertools import chain
from IPython.display import display

batch_size = 128
hidden_layers = [256, 128, 64]
use_l2_reg = False
use_dropout_reg = True
use_learning_rate_decay = False
start_learn_rate = 0.05
decay_steps = 500
decay_by = 0.97

# helper functions
def weight_variable(i, shape):
  #initial = tf.truncated_normal(shape) #default, doesnt work for more than 1 hidden layer (gives nan)
  stddev = np.sqrt(2.0/shape[0])
  initial = tf.truncated_normal(shape, stddev=stddev) #from blog, uses xavier initialization
  return tf.Variable(initial)
  #xavier initalizer (advised as solution to nan problem)
  #return tf.get_variable(name='W'+str(i), shape=shape, initializer=tf.contrib.layers.xavier_initializer())

def bias_variable(shape):
  initial = tf.zeros(shape)
  #initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def calc_nn_logits(var, weights, biases, use_dropout=False): 
    layer_out = tf.matmul(var, weights[0]) + biases[0]
    for i in range(1, len(weights)):
        if use_dropout: #Only on hidden layers input: activations (network cannot rely on any given activation)
             layer_out = tf.nn.dropout(layer_out, keep_prob=0.5)
        layer_out = tf.matmul(
            tf.nn.relu(layer_out), weights[i]
        ) + biases[i]
    return layer_out

def calc_l2_reg(weights):
    return 1e-3 * sum(map(tf.nn.l2_loss, weights))

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Layers
  layers_sizes = [image_size*image_size] + hidden_layers + [num_labels]
  #display(layers_sizes)
    
  # Variables.
  weights = []
  biases = []
  for i in range(len(layers_sizes)-1):
        current_layer_size = layers_sizes[i]
        next_layer_size = layers_sizes[i+1]
        layer_weights = weight_variable(i, [current_layer_size, next_layer_size])
        layer_biases = bias_variable([next_layer_size])
        weights.append(layer_weights)
        biases.append(layer_biases)

  # Training computation.
  logits = calc_nn_logits(tf_train_dataset, weights, biases, use_dropout_reg)
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        )
  if use_l2_reg:
        regulazation = calc_l2_reg(weights)
        loss += regulazation

  # Optimizer.
  if use_learning_rate_decay:
        global_step = tf.Variable(0)  # count the number of steps taken.
        learning_rate = tf.train.exponential_decay(start_learn_rate, global_step, decay_steps, decay_by, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  else:
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(calc_nn_logits(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(calc_nn_logits(tf_test_dataset, weights, biases))

In [ ]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.863114
Minibatch accuracy: 10.2%
Validation accuracy: 8.5%
Minibatch loss at step 500: 0.842699
Minibatch accuracy: 72.7%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 0.656476
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.838550
Minibatch accuracy: 76.6%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.726018
Minibatch accuracy: 78.9%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.554397
Minibatch accuracy: 86.7%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.667634
Minibatch accuracy: 79.7%
Validation accuracy: 84.5%
Minibatch loss at step 3500: 0.553103
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 4000: 0.407349
Minibatch accuracy: 86.7%
Validation accuracy: 85.1%
Minibatch loss at step 4500: 0.593406
Minibatch accuracy: 83.6%
Validation accuracy: 85.2%
Minibatch loss at step 5000: 0.558426
Minibatch accuracy: 87.5%
Validation accuracy

<table style='text-align:center'>
<tr>
<th>Reg</th><th># Steps</th><th># Hidden</th><th>Result1</th><th>Result2</th>
</tr>
<tr bgcolor='#ffff99'>
<td>L2</td><td>30001</td><td>128^3</td><td>95.2%</td><td>95.4%</td>
</tr>
<tr bgcolor='#ffff99'>
<td>-</td><td>30001</td><td>128^3</td><td>94.7%</td><td>94.9%</td>
</tr>
<tr bgcolor='#ffff99'>
<td>Dropout (0.5)</td><td>30001</td><td>256^3</td><td>94.4% (Minibatch 85.9%)</td><td></td>
</tr>
<tr bgcolor='#ffff99'>
<td>Dropout (0.5) + Learn decay</td><td>30001</td><td>256^3</td><td>94.2% (Minibatch 88.3%)</td><td></td>
</tr>
<tr bgcolor='#99ccff'>
<td>L2</td><td>30001</td><td>128^4</td><td>95.2% (Minibatch 93.8%)</td><td>95.3% (Minibatch 94.5%)</td>
</tr>
<tr bgcolor = '#99ccff'>
<td>-</td><td>30001</td><td>128^4</td><td>94.8% (Minibatch 96.9%)</td><td>94.7% (Minibatch 95.3%)</td>
</tr>
<tr bgcolor='#00cc99'>
<td>L2</td><td>4001</td><td>128^4</td><td>93.4%</td><td>93.5%</td>
</tr>
<tr bgcolor='#00cc99'>
<td>-</td><td>4001</td><td>128^4</td><td>93.7%</td><td>93.7%</td>
</tr>
<tr bgcolor='green'>
<td>Dropout (0.5)</td><td>3001</td><td>1024</td><td>92.3% (Minibatch 81.2%)</td><td>92.1% (Minibatch 81.2%)</td>
</tr>
<tr bgcolor='green'>
<td>-</td><td>3001</td><td>1024</td><td>92.8 (Minibatch 82.0%)</td><td>92.7% (Minibatch 82.8%)</td>
</tr>
</table>

<b>Conclusion for L2: </b>Large number of steps on too small training data causes overfitting, L2 prevents that. But with too few steps the same data does not gets retrained so much, and L2 does not help at all.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [59]:
num_steps = 501

small_train_set = train_dataset[:300]
small_train_labels = train_labels[:300]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = small_train_set[offset:(offset + batch_size), :]
    batch_labels = small_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.562675
Minibatch accuracy: 8.6%
Validation accuracy: 19.8%
Test accuracy: 66.6%


<center>Training data size: 300</center>

<table style='text-align:center'>
<tr>
<th>Reg</th><th># Steps</th><th># Hidden</th><th>Result1</th><th>Result2</th>
</tr>
<tr bgcolor='#ffff99'>
<td>L2</td><td>501</td><td>128^3</td><td>83% (Minibatch 100%)</td><td>82.6% (Minibatch 100%)</td>
</tr>
<tr bgcolor='#ffff99'>
<td>-</td><td>501</td><td>128^3</td><td>82.6% (Minibatch 100%)</td><td>82.5% (Minibatch 100%)</td>
</tr>
<tr bgcolor='#ffff99'>
<td>-</td><td>501</td><td>256^3</td><td>83.7% (Minibatch 100%)</td><td>83.3% (Minibatch 100%)</td>
</tr>
<tr bgcolor='#ffff99'>
<td>Dropout (0.5)</td><td>501</td><td>256^3</td><td>83.2% (Minibatch 92.2%)</td><td>83.9% (Minibatch 96.1%)</td>
</tr>
<tr bgcolor='#ffff99'>
<td>L2 + Dropout (0.5)</td><td>501</td><td>256^3</td><td>83.0% (Minibatch 92.2%)</td><td>83.3% (Minibatch 96.1%)</td>
</tr>
<tr bgcolor='green'>
<td>Dropout (0.5)</td><td>501</td><td>1024</td><td>83% (Minibatch 98.4%)</td><td>82.8% (Minibatch 100%)</td>
</tr>
<tr bgcolor='green'>
<td>-</td><td>501</td><td>1024</td><td>83.5% (Minibatch 100%)</td><td>83% (Minibatch 100%)</td>
</tr>
<tr bgcolor='green'>
<td>L2</td><td>501</td><td>1024</td><td>82.7% (Minibatch 100%)</td><td>82.5% (Minibatch 100%)</td>
</tr>
</table>

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Answer
-------
<b>Conclusion: less overfitting: </b> The minibatch accuracy became less than 100%. Although L2 does not seem to have effect here</b>

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


1024